In [2]:
import sys
print(sys.executable) 

/usr/bin/python3


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ! /Users/kshitijahande/opt/anaconda3/envs/pytorch/bin/python -m pip install pandas
# ! /Users/kshitijahande/opt/anaconda3/envs/pytorch/bin/python -m pip install icecream
# ! /Users/kshitijahande/opt/anaconda3/envs/pytorch/bin/python -m pip install nltk
# ! /Users/kshitijahande/opt/anaconda3/envs/pytorch/bin/python -m pip install plotly
# ! /Users/kshitijahande/opt/anaconda3/envs/pytorch/bin/python -m pip install pytorch_pretrained_bert pytorch-nlp
# ! /Users/kshitijahande/opt/anaconda3/envs/pytorch/bin/python -m pip install keras
# ! /Users/kshitijahande/opt/anaconda3/envs/pytorch/bin/python -m pip install tensorflow
# ! /Users/kshitijahande/opt/anaconda3/envs/pytorch/bin/python -m pip install transformers
# ! /Users/kshitijahande/opt/anaconda3/envs/pytorch/bin/python -m pip install ipynb

In [5]:
!pip install ipynb
!pip install plotly
!pip install import-ipynb
!pip install icecream
!pip install transformers

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp37-none-any.whl size=2976 sha256=ea3e126d429a33f28cf0e8588f00b92ab7d08ec71d9777e07d1f65e21b285dc1
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
     |████████████████████████████████| 2.5MB 8.2MB/s 
     |████████████████████████████████| 901kB 47.2MB/s 
     |████████████████████████████████| 3.3MB 46.5MB/s 


In [6]:
import pandas as pd
import numpy as np
import random as rn
from tqdm.notebook import tqdm
from icecream import ic 

import nltk
# from nltk.tokenize import regexp_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

# imports from shudima
import torch
# from pytorch_pretrained_bert import BertModel
# from torch import nn
# from torchnlp.datasets import imdb_dataset
# from pytorch_pretrained_bert import BertTokenizer

# imports from colbert
# import tensorflow as tf
# import bert_tokenization as tokenization
# import tensorflow.keras.backend as K
# from tensorflow import keras
# from transformers import *
from transformers import BertTokenizer

# imports from shudima
# from keras.preprocessing.sequence import pad_sequences
# from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
# from torch.optim import Adam
# from torch.nn.utils import clip_grad_norm_
# from IPython.display import clear_output

# import re
# import os

In [ ]:
# # preprocessing for BERT

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
# tokenizer.tokenize('Hi my nambe is Dima')
# train_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:510] + ['[SEP]'], train['text']))

In [ ]:
# from ipynb.fs.full.utils import *
# from ipynb.fs.full.datasets import *
# from ipynb.fs.full.architectures import *
# from ipynb.fs.full.train_test import *

In [7]:
!pwd
%cd /content/drive/My\ Drive/Colab\ Notebooks/Spring\ 21/NLP/Humour-Detection-2021
!ls
import import_ipynb
from utils import *
from datasets import *
from architectures import *
from train_test import *

/content
/content/drive/My Drive/Colab Notebooks/Spring 21/NLP/Humour-Detection-2021
architectures.ipynb  main_colab.ipynb	    pandas-syntax.ipynb     utils.ipynb
data		     main-dump.ipynb	    README.md
data-analysis.ipynb  model_weights	    results_publictest.csv
datasets.ipynb	     pandas-syntax-2.ipynb  train_test.ipynb
importing Jupyter notebook from utils.ipynb
importing Jupyter notebook from datasets.ipynb
importing Jupyter notebook from architectures.ipynb
importing Jupyter notebook from train_test.ipynb
CrossEntropyLoss()


In [8]:
#init 
train_count = 8000 #8000
# test_count = 
  
MAX_SENTENCE_LENGTH = 128 #colbert=20
MAX_SENTENCES = 5
MAX_LENGTH = 100

In [9]:
#===============================================================================
# Device Selction (GPU/CPU) and Seed Initialization
#===============================================================================
# args parameters
no_cuda=False
seed =1
test_model = False

ic(torch.cuda.is_available())
use_cuda = not no_cuda and torch.cuda.is_available()
device   = torch.device("cuda" if use_cuda else "cpu")
kwargs   = {'num_workers': 32, 'pin_memory': True} if use_cuda else {}

torch.manual_seed(seed)
if(use_cuda):
	torch.cuda.manual_seed(seed)
	torch.backends.cudnn.benchmark = True
np.random.seed(seed)

print("\nDevice: " + str(device) +"; Seed: "+str(seed))


ic| torch.cuda.is_available(): True



Device: cuda; Seed: 1


In [10]:
#===============================================================================
# Dataset and Model Selection
#===============================================================================

# if seq length is 128 batch size should be 32
traindata = torch.utils.data.DataLoader(dataset= Hahackathon("data/train.csv"
, basenet= 'bert')
, batch_size= 32, shuffle= True,  **kwargs)

valdata = torch.utils.data.DataLoader(dataset= Hahackathon("data/dev.csv"  
, basenet= 'bert' )
, batch_size= 128, shuffle= False, **kwargs)

testdata = torch.utils.data.DataLoader(dataset= Hahackathon("data/gold-test-27446.csv" 
, basenet= 'bert', is_test= True) 
, batch_size= 128, shuffle= False, **kwargs)

# ic("Size of the dataset",len(traindata.dataset))
model = multitask_fc('bert')

if torch.cuda.device_count() > 1:
	print("Using {} GPUs! \n".format(torch.cuda.device_count()))
	model = torch.nn.DataParallel(model)
 
model = model.to(device)


Sampled input from the file: data/train.csv
   id  ... offense_rating
0   1  ...            0.2
1   2  ...            1.1
2   3  ...            2.4
3   4  ...            0.0
4   5  ...            0.1

[5 rows x 6 columns]
Tokenizer: bert-base-uncased



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Sampled input from the file: data/dev.csv
     id  ... offense_rating
0  8001  ...           1.70
1  8002  ...           0.00
2  8003  ...           1.15
3  8004  ...           3.75
4  8005  ...           2.25

[5 rows x 6 columns]
Tokenizer: bert-base-uncased



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Sampled input from the file: data/gold-test-27446.csv
     id  ... offense_rating
0  9001  ...           0.90
1  9002  ...           0.35
2  9003  ...           0.10
3  9004  ...           0.00
4  9005  ...           0.35

[5 rows x 6 columns]
Tokenizer: bert-base-uncased



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Base architecture: bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero

In [11]:

from sklearn.metrics import roc_auc_score, mean_squared_error, f1_score, accuracy_score
 
# mse_criterion = torch.nn.MSELoss()
ce_criterion  = torch.nn.CrossEntropyLoss()
ic(ce_criterion)

def concat_output1(output, device):
    temp = torch.tensor([]).to(device)

    if output.shape[1] == 2:
        ic()
        temp = torch.argmax(output[:, 0:2], dim=1).view(-1,1)
        ic(temp)
        return temp

    temp = torch.argmax(output[:, 0:2], dim=1).view(-1,1)
    temp = torch.cat((temp, torch.argmax(output[:, 2:4], dim= 1).view(-1,1)), dim= 1)
    temp = torch.cat((temp, output[:, 4].view(-1,1)), dim= 1)
    temp = torch.cat((temp, output[:, 5].view(-1,1)), dim= 1)
    return temp

def loss_func1(output, target, device):

    humor_id = (target[:, 0]==1)
    target   = target.float()
    # ic(output[:, 0:2])
    
    loss1 = ce_criterion(output[:, 0:2], target[:, 0].long())
    loss=loss1
    
    # this block of generating temp is verified throughly
    temp = concat_output1(output, device)
    # ic('outside concat_output ', temp)
    temp = temp.detach()

    return loss, temp

def train1(model, dataloader, optimizer, device):
#     ic('1')
    model.train()
#     ic('2')
    train_loss = AverageMeter()

    for batch_id, batch_data in enumerate(dataloader):
        data       = batch_data[0]
        target     = batch_data[1].to(device)
#         ic(data)
#         ic(target)

        input_id   = data[0].to(device).squeeze()
        mask_id    = data[1].to(device).squeeze()
        segment_id = data[2].to(device).squeeze()
#         ic(input_id)
#         ic(mask_id)
#         ic(segment_id)
        
        output = model(input_id, mask_id, segment_id)
#         ic('output model')
        
        loss, _ = loss_func1(output, target, device= device)
        # ic('outside loss_func')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss.update(loss.item(), target.shape[0])

    return train_loss.avg

def test1(model, dataloader, device):
	model.eval()
	test_loss = AverageMeter()
	outputs   = torch.tensor([]).to(device)
	targets   = torch.tensor([]).to(device)

	with torch.no_grad():
		for batch_id, batch_data in enumerate(dataloader):
			data       = batch_data[0]
			target     = batch_data[1].to(device)

			token_id   = data[0].to(device).squeeze()
			mask_id    = data[1].to(device).squeeze()
			segment_id = data[2].to(device).squeeze()

			output     = model(token_id, mask_id, segment_id)

			loss, temp = loss_func1(output, target, device= device)
			outputs    = torch.cat((outputs, temp)  , 0)
			targets    = torch.cat((targets, target), 0)

			test_loss.update(loss.item(), target.shape[0])

	outputs = outputs.cpu().numpy()
	targets = targets.cpu().numpy()

	humor_fscore          = f1_score          (targets[:, 0], outputs[:, 0])
	# controversy_fscore    = f1_score          (targets[:, 1], outputs[:, 1])
	humor_acc             = accuracy_score    (targets[:, 0], outputs[:, 0])
	# controversy_acc       = accuracy_score    (targets[:, 1], outputs[:, 1])
	# humor_rating_rmse     = mean_squared_error(targets[:, 2], outputs[:, 2], squared= False)
	# offensive_rating_rmse = mean_squared_error(targets[:, 3], outputs[:, 3], squared= False)

	return test_loss.avg, [humor_fscore, humor_acc] #, controversy_fscore, controversy_acc, humor_rating_rmse, offensive_rating_rmse]

def get_result_on_evaldata1(model, dataloader, filename, device):
	model.eval()
	outputs  = torch.tensor([]).to(device)
	text_ids = torch.tensor([]).to(device)

	with torch.no_grad():
		for batch_id, batch_data in enumerate(dataloader):
			data       = batch_data[0]
			text_id    = batch_data[1].to(device)

			token_id   = data[0].to(device).squeeze()
			mask_id    = data[1].to(device).squeeze()
			segment_id = data[2].to(device).squeeze()

			output = model(token_id, mask_id, segment_id)

			temp     = concat_output1(output, device)
			outputs  = torch.cat((outputs, temp), 0)
			text_ids = torch.cat((text_ids, text_id), 0)
	
	ic(outputs.shape, text_ids.shape)
	outputs  = outputs.cpu().numpy()
	text_ids = text_ids.cpu().numpy()

	results = np.hstack((text_ids, outputs))
	df      = pd.DataFrame(data    = results,  
	                       columns = ['id', 'is_humor', 'humor_controversy', 'humor_rating', 'offense_rating']) 

	df['id']                = (df['id']).astype('int')
	df['is_humor']          = (df['is_humor']).astype('int')
	# df['humor_controversy'] = (df['humor_controversy']).astype('int')
	df = df[['id', 'is_humor']]   #, 'humor_rating', 'humor_controversy', 'offense_rating']]
	
	df.to_csv(filename, index = False)
	ic("Result file is saved to {}".format(filename))


ic| ce_criterion: CrossEntropyLoss()


In [18]:
learning_rate=0.00002 #0.01 to 0.00002
start_epoch = 1
epochs = 30 #10 to 30
train_loss_arr = []
test_loss_arr = []
metrics_arr = [] #f1 , accuracy

if not test_model:

  ic("Training starts")
  optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)

  for ep in range(start_epoch, start_epoch+epochs):
      train_loss = train1(model, traindata, optimizer, device)
      test_loss, metrics = test1(model, valdata, device)

      ic(train_loss)
      train_loss_arr.append(train_loss)
      test_loss_arr.append(test_loss)
      metrics_arr.append(metrics)
      # ic('Loss/train', train_loss, ep)
      # ic('Loss/val', test_loss , ep)
      # ic('is_humor_f1/val', metrics[0], ep)
      # ic('is_humor_acc/val', metrics[1], ep)

ic| 'Training starts'
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning:

This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



KeyboardInterrupt: ignored

In [15]:
ep = 30 #epochs
run_iter = 2 #iterations you ran the model
save_dir = os.path.join('model_weights', 'run_{}'.format(run_iter))
create_folder(save_dir)
save_path = os.path.join(save_dir, "epoch_{}".format(ep) + ".pth")
save_model(model, optimizer, opt= None, epoch= ep, save_file= save_path)

Creating directory model_weights/run_2


In [17]:
ic("Generating prediction file for post-evaluation phase")
filename   = "results_publictest_2.csv"
model = model.to(device)
get_result_on_evaldata1(model, testdata, filename, device= device)

ic| 'Generating prediction file for post-evaluation phase'
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning:

This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.

ic| outputs.shape: torch.Size([1000, 4])
    text_ids.shape: torch.Size([1000, 1])
ic| "Result file is saved to {}".format(filename): 'Result file is saved to results_publictest_2.csv'


In [19]:
ic(test_loss_arr)

ic| test_loss_arr: []


[]

In [13]:
ic(metrics_arr)
ic(test_loss_arr)
ic(train_loss_arr)
print('Final train loss: ', train_loss_arr[-1])
print(' test loss: ', test_loss_arr[-1])
print('F1 score: ', metrics_arr[-1][0])
print(' Accuracy: ', metrics_arr[-1][1])

import plotly.graph_objects as go

x_axis = list(range(1, len(train_loss_arr)+1)) #epochs
y_axis = list(train_loss_arr)

fig = go.Figure(data = go.Scatter(x= x_axis, y= y_axis))
fig.update_layout(title='train loss',
                   xaxis_title='epochs',
                   yaxis_title='loss')
fig.show(renderer="colab")

y2_axis = np.array(metrics_arr)
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x= x_axis, y= y2_axis[:,0],
                    mode='lines',
                    name='f1-score'))
fig2.add_trace(go.Scatter(x= x_axis, y= y2_axis[:,1],
                    mode='lines+markers',
                    name='accuracy'))
fig2.show(renderer="colab")

ic| metrics_arr: [[0.9186228482003129, 0.896],
                  [0.930739299610895, 0.911],
                  [0.9172077922077922, 0.898],
                  [0.918429003021148, 0.892],
                  [0.9023323615160349, 0.866],
                  [0.9200930954228085, 0.897],
                  [0.9271317829457365, 0.906],
                  [0.9229547259729945, 0.903],
                  [0.9182098765432098, 0.894],
                  [0.9227202472952086, 0.9],
                  [0.9163346613545817, 0.895],
                  [0.9177018633540373, 0.894],
                  [0.9059443190368699, 0.875],
                  [0.9158453373768006, 0.889],
                  [0.926829268292683, 0.904],
                  [0.9215384615384615, 0.898],
                  [0.9206842923794712, 0.898],
                  [0.9191685912240184, 0.895],
                  [0.9195046439628484, 0.896],
                  [0.9086651053864169, 0.883],
                  [0.9174311926605505, 0.892],
                  

Final train loss:  9.26312994415639e-05
 test loss:  0.7476667199134827
F1 score:  0.9158301158301159
 Accuracy:  0.891
